<a href="https://colab.research.google.com/github/DiegoSReco/Labor-Market-Analysis-ENOE/blob/main/Analisis_de_Encuestas_en_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# **Taller de Análisis de Encuestas Complejas en R**
## **Colegio de Economistas de Aguascalientes**
## **Diego Sánchez Rojas**

### [LinkedIn](https://www.linkedin.com/in/diegorojas12/) [Github](https://github.com/DiegoSReco)


**Objetivo:**





# **1. Introducción**
- Los gobiernos, las empresas, ONGs e investigadores necesitamos información de características, opiniones, preferencias y comportamiento de una población.
- Por la restricción de tiempo y dinero recurrimos al uso de **encuestas complejas** o  **encuestas de diseño probabilístico.**

###**1.1 ¿Qué son las encuestas complejas?**

- Las encuestas complejas tienen un diseño de muestra probabilístico, en donde los encuestados son seleccionados de forma aleatoria de tal forma que los resultados representen de forma precisa y eficiente a la población de interés.


sampling-inference.svg




###**1.2 Tipos de muestreo probabilístico**
El muestreo probabilístico se caracteriza porque todos los individuos de la población tienen una probabilidad conocida y distinta de cero de ser seleccionados. Esto permite hacer inferencias estadísticas sobre la población.

- **Muestreo aleatorio simple**: Cada miembro de la población tiene la misma probabilidad de ser seleccionado.

- **Muestreo sistemático**: Se selecciona un punto de partida aleatorio y luego se elige cada k-ésimo miembro de la población.

- **Muestreo estratificado**: La población se divide en subgrupos o estratos (por ejemplo, género, edad, estados) y se selecciona una muestra aleatoria de cada estrato.

- **Muestreo por clusters**: La población se divide en grupos o clusters (como barrios, ciudades), y se seleccionan aleatoriamente algunos clusters. Luego, se estudian todos los individuos dentro de los clusters seleccionados o se toma una muestra dentro de ellos.

- **Muestreo multietápico**: Combina varios tipos de muestreo (por ejemplo, clusters y luego aleatorio) en diferentes etapas.


**Bibliografía:**

- Lehtonen, R. & Pahkinen E. (2004). Practical Methods for Design and Analysis of Complex Surveys

- Levy, P. S., & Lemeshow, S. (1993). Sampling of Populations: Methods and Applications.



### **1.3 Conceptos básicos de una encuesta compleja**

- **Estrato**

 Subgrupo o subdivisión de la población que comparte ciertas características comunes. En el muestreo estratificado, la población se divide en estratos, como grupos de edad, género, ubicación geográfica, nivel socioeconómico, entre otros. Cada estrato se trata como una subpoblación independiente, y se toma una muestra aleatoria de cada uno.

- **Unidad primaria de muestreo (UPM)**
  Primer nivel de selección de la muestra en un muestreo por etapas. En una encuesta compleja, especialmente en un muestreo por clusters o multietápico, se seleccionan primero las UPMs, que son subgrupos más grandes de la población, y luego se eligen unidades de muestreo adicionales dentro de ellas.  Ejemplo de UPM sería las viviendas.


  - **Ponderadores**
  Son factores aplicados a cada observación en la muestra para corregir las probabilidades de selección y asegurar que los resultados sean representativos de la población total.
  Ejemplo de cálculo de ponderadores:
  
  Si suponemos que el muestreo fue aletorio simple la probabilidad de selección será la misma y la definimos como $p_i$, el peso o ponderador será igual al inverso de la probabilidad:
  $$ Peso Base (w_i) = \frac{1}{p_i}$$

  Población: 10,000 personas

  Muestra seleccionada: 500 personas  

$$p_i = \frac{500}{10,000} = 0.05 $$

$$\therefore~~~w_i = \frac{1}{0.05}=20$$



- En la realidad no es tan simple y se deben considerar ajustes por no respuesta, ajuste porestratificación o clusterisación, normalización de ponderadores, etc.



# **¿Cómo podemos manejar el diseño de la encuesta?**

*   Anteriormente se utilizaban softwares como SPSS y Stata
*   Software libre y de código abierto como R

### **Ventajas**


1.  A diferencia de SPSS y Stata que requieren licencias costosas, R es completamente gratuito. Esto facilita el acceso para estudiantes, investigadores y organizaciones con presupuestos limitados.

2. Flexibilidad y facilidad:
En R, específicamente utilizamos la  paquete **srvyr** la cual combina la sintaxis intuitiva de **dplyr** para manipulación de datos con las capacidades de **survey** para análisis de encuestas complejas.


3. Reproducibilidad y automatización: Permite crear flujos de trabajo reproducibles y personalizados debido a la integración con otros paquetes de R para visualizaciónes de gran calidad (**ggplot2**) y reportes automatizados (**RMarkdown o Quarto**).

[dplyr](https://dplyr.tidyverse.org/index.html)

[srvyr](https://cran.r-project.org/web/packages/srvyr/index.html)








# **2. Ejemplo**

## **¿Cuál es el promedio de horas trabajadas por semana en México, y cómo cambia por entidad y sexo?**

*   **Utilizaremos los microdatos de la Encuesta Nacional de Ocupación y Empleo (ENOE) trimestre II de 2024 del INEGI**.
*   **Específicamente, la tabla de Sociodemográficos.**



## **2.1 Carga de Datos**

*  **Realizamos la descarga temporal de los datos de la página del INEGI:**

In [ ]:
#Librerias
install.packages('pacman')
library('pacman')

p_load('tidyverse',
        "readxl",
        "data.table",
        "curl")

In [ ]:
#Datos
#Link ENOE 2024 T2
url_enoe  <- "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos//enoe_2024_trim2_csv.zip"
#Descarga temporal
td  <- tempdir() #directorio
tf <- tempfile(tmpdir= td, fileext = ".zip") #file
dir.create(td, recursive = TRUE, showWarnings = FALSE)
#Descarga temporal
curl::curl_download(url_enoe, tf, quiet = FALSE, mode = "wb")
#Nombres de los file dentro del archivo que descargamos temporalmente
zipped_files <- as.data.frame(unzip(tf, list = TRUE,  overwrite=TRUE))$Name
#UTILIZAMOS TABLA DE SOCIODEMOGRÁFICOS
file_to_load <- zipped_files[grepl("(?i)sdem", zipped_files)]
#Unzip
unzip(tf, files= file_to_load, exdir=td, overwrite=TRUE)
#path del file que utilizaremos
fpath <- file.path(td, file_to_load  )
#Unlink
unlink(td)

print(file_to_load)


[1] "ENOE_COE1T224.csv" "ENOE_COE2T224.csv" "ENOE_HOGT224.csv" 
[4] "ENOE_SDEMT224.csv" "ENOE_VIVT224.csv" 


### Vistazo de los datos que usaremos

In [ ]:
  #Cargamos datos con la función fread de la libreria data.table
  #Load file
  df_enoe <- fread(fpath)
  glimpse(df_enoe)


Rows: 427,123
Columns: 114
$ r_def      <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ loc        <lgl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA,…
$ mun        <int> 2, 2, 10, 6, 6, 6, 6, 6, 11, 11, 10, 13, 5, 7, 7, 15, 14, 1…
$ est        <int> 10, 10, 30, 20, 20, 20, 30, 30, 20, 20, 30, 20, 30, 20, 30,…
$ est_d_tri  <int> 122, 122, 124, 123, 123, 123, 124, 124, 123, 123, 124, 123,…
$ est_d_men  <int> NA, NA, NA, 117, 117, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ ageb       <int> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ t_loc_tri  <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ t_loc_men  <int> NA, NA, NA, 1, 1, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 1…
$ cd_a       <int> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ ent        <int> 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,…
$ con        <int> 40001, 40001, 40002, 40003, 40003, 40003, 40004, 40004, 400…
$ upm        

## Preparación de los datos

In [ ]:
#Preparación de datos
#Cambiamos nombres de algunas variables
rename_dict <- c( "clase1" = "clas_PEA",
                  "clase2" = "clas_PO",
                  "emp_ppal" = "clas_INF",
                  "sex" = "sexo",
                  "hrsocup" = "n_hrs",
                  "ingocup" = "ing_men",
                  "ing_x_hrs" = "ing_hrs",
                  "t_loc" = "t_loc",
                  "ent" = "entidad")

colnames(df_enoe) <- rename_dict[match(colnames(df_enoe),
                                                   names(rename_dict))]


In [ ]:
### Filtros y codificación de algunas variables

df_enoe_f <- df_enoe |>
             filter(r_def==0 & (c_res ==1 | c_res ==3) & (eda >= 15 &  eda< 30)) |>    #c_res: Condición de residencia y r_def: residencia definitiva. Para el uso de variables tenemos que aplicar este criterio según el Descriptor del archivo
             mutate(
                     clas_ocupado = case_when(
                                  clas_PO == 1 ~ 'ocupados' ,
                                  clas_PO == 2 ~ 'desocupados',
                                  clas_PO == 3 ~ 'disponibles',
                                  clas_PO == 4 ~ 'no disponibles',
                                  TRUE ~ NA),

                      sex = case_when(sexo == 1 ~ "Hombre",
                                      sexo == 2 ~ "Mujer",
                                        TRUE ~ NA ),

                      entidad_ab = case_when(
                                               entidad == 1 ~ "AGS",
                                               entidad == 2 ~ "BC",
                                               entidad == 3 ~ "BCS",
                                               entidad == 4 ~ "CAMP",
                                               entidad == 5 ~ "COAH",
                                               entidad == 6 ~ "COL",
                                               entidad == 7 ~ "CHIS",
                                               entidad == 8 ~ "CHIH",
                                               entidad == 9 ~ "CDMX",
                                               entidad == 10 ~ "DGO",
                                               entidad == 11 ~ "GTO",
                                               entidad == 12 ~ "GRO",
                                               entidad == 13 ~ "HGO",
                                               entidad == 14 ~ "JAL",
                                               entidad == 15 ~ "MEX",
                                               entidad == 16 ~ "MICH",
                                               entidad == 17 ~ "MOR",
                                               entidad == 18 ~ "NAY",
                                               entidad == 19 ~ "NL",
                                               entidad == 20 ~ "OAX",
                                               entidad == 21 ~ "PUE",
                                               entidad == 22 ~ "QRO",
                                               entidad == 23 ~ "QR",
                                               entidad == 24 ~ "SLP",
                                               entidad == 25 ~ "SIN",
                                               entidad == 26 ~ "SON",
                                               entidad == 27 ~ "TAB",
                                               entidad == 28 ~ "TAMS",
                                               entidad == 29 ~ "TLAX",
                                               entidad == 30 ~ "VER",
                                               entidad == 31 ~ "YUC",
                                               entidad == 32 ~ "ZAC",
                                               TRUE ~ "Desconocido") )   |>


                          filter(clas_PO == 1 )  #Me quedo con los ocupados


## **Paquetería srvyr**

Combina la capacidad de la paquetería survery con la sintaxis intuitiva de dplyr

**Funciones:**

as_survey_design(): Convierte un dataframe en un objeto de diseño de encuesta


**Funciones de resumen y estadísticas:**

survey_mean(): Calcula medias ponderadas

survey_total(): Calcula totales ponderados

survey_ratio(): Calcula razones/proporciones

survey_quantile(): Calcula cuantiles ponderados

survey_median(): Calcula la mediana ponderada

survey_var(): Calcula varianzas muestrales




In [ ]:
#Cargamos libreria
p_load('srvyr')

#UPM: Unidad primaria de muestreo
#EST: Estrato
#fac_tri: Factor de expansión trimestrar

# Diseño en srvyr
df_enoe_d <- df_enoe_f |>
                as_survey_design(
                    ids = upm,                  # ID
                    strata = est,               # Estrato
                    weights = fac_tri,           # Pesos de cada observación
                    nest =TRUE)                  #Cada UPM está contenida en un solo estrato, entonces los estratos son anidados (nest) dentro de los conglomerados.



In [ ]:
#Promedio de horas semanales trabajadas
df_enoe_d  |>
  summarise(prom_horas = survey_mean(n_hrs )) #vartype = "ci")

prom_horas,prom_horas_se
<dbl>,<dbl>
40.63572,0.1437471


In [ ]:
#Promedio de horas semanales trabajdas por entidad
table_horas_ent <- df_enoe_d  |>
                   group_by(entidad_ab)  |>
                    summarise(survey_mean(n_hrs))  |>
                      arrange(desc(coef))

table_horas_ent

entidad_ab,coef,_se
<chr>,<dbl>,<dbl>
AGS,43.43387,0.4290063
BC,40.29964,0.4376979
BCS,41.32339,0.6749899
CAMP,40.45934,0.7223984
CDMX,39.57450,0.6819632
CHIH,39.84181,0.4711774
CHIS,41.59329,0.6937295
COAH,41.08718,0.4091506
COL,39.37620,0.7693841


In [ ]:
#Promedio de horas semanales trabajdas por entidad y sexo
df_enoe_d  |>
  group_by(entidad_ab, sex)  |>
     summarise(survey_mean(n_hrs))

entidad_ab,sex,coef,_se
<chr>,<chr>,<dbl>,<dbl>
AGS,Hombre,46.12230,0.5219736
AGS,Mujer,39.43186,0.5496093
BC,Hombre,41.98870,0.4796051
BC,Mujer,37.80203,0.6647691
BCS,Hombre,44.17163,0.7326871
BCS,Mujer,37.15739,1.0119375
CAMP,Hombre,43.38589,0.9779845
CAMP,Mujer,35.56918,0.8107598
CDMX,Hombre,41.94775,0.9412069
